# Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.




In [1]:
# Импортируем библиотеки
# прочитаем файл и посмотрим информацию о датасете


import pandas as pd 
data = pd.read_csv ("C:\\Users\\Пользователь\\Documents\\Data analyst\\СПРИНТ 3\\data.csv")
print (data.head(10))
data.info()               



   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

## Предобработка данных. 
Изучим датасет на пропуски и дубликаты.

In [2]:
total_values = data ['children'].count()    #определил количество значений, сохранил в переменную
nan = data['days_employed'].isna().sum()    #определил количество пропущенных значений в столбце 'days_employed'
print (data['days_employed'].isna().mean())
print (total_values)
print (nan)
  

0.10099883855981417
21525
2174


Нашли долю пропусков. Она достаточно большая, поэтому удалить строки с пропусками нельзя.

In [3]:
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
data.info()    #рассчитали медианные значения столбцов с пропусками, 
               #сгруппировав данные по income_type, заполнили пропуски получившимися значениями. методом info() проверили, удалось ли избапвться от пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
#меняем тип данных в столбцах
data['days_employed'] = data['days_employed'].astype('int64').round()     
data['total_income'] = data['total_income'].astype('int64')
data.info()
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2 

In [5]:
#Проверим количество дубликатов и регистр написания:
display(data['days_employed'].value_counts())
display(data['education'].value_counts())
display(data['family_status'].value_counts())
display(data['income_type'].value_counts())
display(data['purpose'].value_counts())
display(data['children'].value_counts())

#Подведем значения столбца с образованием под строчный регистр:
data['education']=data['education'].str.lower()
#Заменим значение количества детей с -1 на 1:
data['children'] = data['children'].replace(-1, 1)
#Заменим значение количества детей с 20 на 2:
data['children'] = data['children'].replace(20, 2)

#Посчитем общее количество дубликатов в таблице:
data.duplicated().sum()
data = data.drop_duplicates()
data.reset_index(drop=True)


-1574      1106
-1547       512
 365213     414
-2689       154
-133         16
           ... 
-9090         1
 360849       1
-2101         1
 380864       1
 343937       1
Name: days_employed, Length: 9086, dtype: int64

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [6]:
 #сделали все значения в столбце положительными
data['days_employed'] = data['days_employed'].abs() 
#в столбце есть данные, похожие на выбросы. Причиной может быть разные информационные системы исходных данных.
#вероятно такие больште цифры получились из-за того,что стаж записан в часах.
data.loc[data['days_employed'] >=17155, 'days_employed'] =  data.loc[data['days_employed'] >=17155, 'days_employed']/24  #цифру в 17155 определили как мкасимальноьвозможный трудовой стаж
display (data.head(10))                                                                                                 # с 18 до 65 лет.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.00,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.00,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.00,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.00,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.75,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.00,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.00,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.00,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.00,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.00,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [7]:
data= data.drop(data[data['children'] == -1].index)
data= data.drop(data[data['children'] == 20].index)
display(data['children'].value_counts())



0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

## Исследовательский анализ

In [8]:
# создадим сводную таблицу для образования
data_education= data.pivot_table(index='education', values='education_id')
data_education



,education_id
education,
высшее,0
начальное,3
неоконченное высшее,2
среднее,1
ученая степень,4


In [9]:
# создадим сводную таблицу для семейного статуса
data_family= data.pivot_table(index='family_status', values= 'family_status_id')
data_family

,family_status_id
family_status,
Не женат / не замужем,4
в разводе,3
вдовец / вдова,2
гражданский брак,1
женат / замужем,0


In [10]:
data.drop(columns=['education', 'family_status'], axis=1, inplace=True)

In [11]:
# Функция для заполнения ячеек в новом столбце.
#напишем функцию, которая в зависимости от размера дохода присвоит категорию A-E и заполнит ячейки в новом столбце total_income_category
# в категорию Е попадают заемщики с доходом от 0 до 30000р.
# в категорию D попадают заемщики с доходом от 30 001 до 50 000р.
# в категорию С попадают заемщики с доходом от 50 001 до 200 000р.
# в категорию В попадают заемщики с доходом от 200 001 до 1 000 000р.
# в категорию А попадают заемщики с доходом свыше 1 000 000 р.

def income_cat (income):
    if income <= 30000:
        return 'E'
    if income <= 50000 and income >= 30001:
        return 'D'
    if income >= 50001 and income <= 200000:
        return 'C'
    if income >= 200001 and income <= 1000000:
        return 'B'    
    else:
        return 'A'
    

data['total_income_category']= data['total_income'].apply(income_cat)
data.head(10)
    
    

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.00,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.00,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.00,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.00,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.75,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.00,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.00,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.00,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.00,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.00,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


In [12]:
#проверим,что получилось
data['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

In [13]:
# найдем все уникальные значения в столбце целей кредита, чтобы потом их можно было бы разбить на категории
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [24]:
# создадим функцию, которая категоризирует цели кредита
def purpose_cat (row):
   
    if row == 'покупка жилья' or row == 'операции с жильем' or row == 'покупка жилья для семьи' or row == 'покупка недвижимости'\
or row == 'покупка коммерческой недвижимости' or row == 'покупка жилой недвижимости' or row == 'строительство собственной недвижимости'\
or row == 'недвижимость' or row == 'строительство недвижимости' or row == 'операции с коммерческой недвижимостью' or row == 'строительство жилой недвижимости'\
or row == 'жилье' or row == 'операции со своей недвижимостью' or row == 'покупка своего жилья' or row == 'операции с недвижимостью'\
or row == 'покупка жилья для сдачи':
        return 'операции с недвижимостью'
    if row == 'приобретение автомобиля' or row == 'на покупку подержанного автомобиля' or row == 'на покупку своего автомобиля' or row == 'автомобили'\
or row == 'сделка с подержанным автомобилем' or row == 'автомобиль' or row == 'свой автомобиль'\
or row == 'сделка с автомобилем' or row == 'на покупку автомобиля':
        return 'операции с автомобилем'
    if row == 'дополнительное образование' or row == 'образование' or row == 'заняться образованием' or row == 'получение образования'\
or row == 'получение дополнительного образования' or row == 'получение высшего образования' or row == 'профильное образование'\
or row == 'высшее образование' or row == 'заняться высшим образованием':
        return 'получение образования'
    return 'проведение свадьбы'

#создадим новый столбец с категориями целей кредита, которые получили функцией.
data['purpose_category']=data['purpose'].apply(purpose_cat)
display (data.head(10))


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.00,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.00,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.00,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.00,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.75,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.00,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.00,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.00,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.00,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.00,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [ ]:
#проанализируем зависимость возврата кредита от количества детей. Для этот составим сводную таблицу
pivot_table_children = data.pivot_table(index='children', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_children['percent'] = pivot_table_children[1]/(pivot_table_children[1] \
                                                                       + pivot_table_children[0]) * 100
display (pivot_table_children.head(10))

# из таблицы видим, что наиболее надежными заемщиками являются люди, не имеющие детей. Процент долгов по кредиту 7.5,
# самый большой процент задолженности среди заемщиков с 4 детьми. Он составляет 9.76 %.

In [ ]:
#проанализируем зависимость возврата кредита от количества детей. Для этот составим сводную таблицу
pivot_table_family = data.pivot_table(index='family_status_id', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_family['percent'] = pivot_table_family[1]/(pivot_table_family[1] \
                                                                       + pivot_table_family[0]) * 100
display (pivot_table_family.head(10))

# из таблицы видим, что вдовцы в целом наиболее надежные заемщики. Процент задолженоости составляет 6.56 %
# самые ненадежные - неженатые. Процент задолженности людей в гражданском браке 9.35 %, женатых - 7.55%, в разводе - 7.11%

In [29]:
pivot_table_purpose_category = data.pivot_table(index='purpose_category', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_purpose_category['percent'] = pivot_table_purpose_category[1]/(pivot_table_purpose_category[1] \
                                                                       + pivot_table_purpose_category[0]) * 100
display (pivot_table_purpose_category)

# из таблицы видим,что самый низкий процент невозврата в категироо "операции с недвижимостью" - 7.32%
# самая "рисковая" категория оказалась "операции с автомобилем" - 9.35%

debt,0,1,percent
purpose_category,,,
операции с автомобилем,3903,403,9.359034
операции с недвижимостью,9457,747,7.320659
получение образования,3643,370,9.220035
проведение свадьбы,2710,221,7.540089


In [ ]:
# исследуем зависимость просроченной задолженности от уровня дохода. 
pivot_table_income = data.pivot_table(index='total_income_category', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_income['percent'] = pivot_table_income[1]/(pivot_table_income[1] \
                                                                       + pivot_table_income[0]) * 100
display (pivot_table_income)
#Самый низкий % невозвратов среди заемщиков категории D - всего 6%
#Заемщики кат. С и Е наименее надежны с процентами задолженности 8.49 и 9 соответственно. 
#Категриии А и В имеют показатели невозврата в 8% и 7.06% соответственно.

## Общий вывод
Исследование показало зависимость надежности заемщика от его семейного положения, уровня дохода, количества детей и целей кредита.

самый надежный аватар заемщика :вдовцы, без детей, с доходами от  30 001 до 50 000р. с целью кредита - на недвижимость.
самым ненадежным заемщиком будет: многодетный родитель с 4 детьми, в гражданском браке, с уровнем дохода до 30 000р. и целью кредита - операции с автомобилем.